In [1]:
import tensorflow as tf
from utils import *
from bicond_tf10 import create_bicond_embeddings_reader
import time
import subprocess
from gensim.models import word2vec
import model_condBiRNN_wdev

config = model_condBiRNN_wdev.Config()
config.debug = False
config.epochs = 8
model = model_condBiRNN_wdev.biRNN_model(config)

init = tf.global_variables_initializer()

Loading word2vec data ...
Loading train data ...
Loading dev data ...
Loading test data ...


In [2]:
with tf.Session() as session_tf:
    session_tf.run(init)

    # classifying lables for the true stage_2 data in order to measure stage_2
    # perforamnce independently
    labels_pred = model.predict(
        session_tf,
        model.encoded_targets_test,
        model.encoded_tweets_test)
    output = model.test_data_evaluation(session_tf)

In [3]:
# accuracy
labels_gr = []

for label in model.labels_test:
    if label == u'NONE':
        labels_gr.append(0)
    elif label == u'AGAINST':
        labels_gr.append(1)
    elif label == u'FAVOR':
        labels_gr.append(2)

accuracies = []
for idx in xrange(len(labels_pred)):
    if labels_pred[idx] == labels_gr[idx]:
        accuracies.append(1)
    else:
        accuracies.append(0)
accuracy = float(sum(accuracies)) / len(accuracies)
print(accuracy)

0.429985855728


In [4]:
res = [0.427157001414, 0.431400282885, 0.428571428571, 0.432814710042, 0.435643564356]

In [20]:
np.mean(res)

0.43111739745359995

In [21]:
np.std(res)

0.0030203898874174373

In [6]:
print(output)



Results				 
FAVOR     precision: 0.6667 recall: 0.3919 f-score: 0.4936
AGAINST   precision: 0.5904 recall: 0.4916 f-score: 0.5365
------------
Macro F: 0.5151




In [7]:
model.dev_data_size

1876

In [8]:
model.train_data_size

16884

In [9]:
model.test_data_size

707

In [10]:
707. / 19467

0.03631787126932758

In [11]:
1876 + 16884

18760

In [12]:
1876. / 19467

0.09636821287306724

In [13]:
len(filter(lambda x: x[1] == 1, model.encoded_labels_test))

299

In [14]:
with tf.Session() as session_tf:
    session_tf.run(init)
    model.saver.restore(session_tf, model.config.tf_saver_folder +
                       model.config.model_name)
    keep_prob = 1.0
    targets_lens = np.array(
        model.config.source_len * np.ones(model.test_data_size),
        dtype=model.config.int_type_np
    )
    tweets_lens = np.array(
        model.config.target_len * np.ones(model.test_data_size),
        dtype=model.config.int_type_np
    )
    predicted_probs = session_tf.run(
        model.probs,
        feed_dict={
            model.source_placeholder: model.encoded_targets_test,
            model.source_len_placeholder: targets_lens,
            model.target_placeholder: model.encoded_tweets_test,
            model.target_len_placeholder: tweets_lens,
            model.keep_prob_placeholder: keep_prob
        }
    )
    '''Postprocessing of predicted_labels in order to refine 'NONE'.
    If a tweet has one of the keywords for Trump, its predicted label is
    forced to be AGAINST/FAVOR acoording to the predicted distribution.'''
    predicted_labels = np.argmax(predicted_probs, axis=1)
    target_keywords = ['donald trump', 'trump', 'donald']
    for idx, tweet in enumerate(model.tweets_test):
        for key in target_keywords:
            if key.lower() in tweet.lower():
                if predicted_probs[idx][1] > predicted_probs[idx][2]:
                    predicted_labels[idx] = 1
                else:
                    predicted_labels[idx] = 2
                break

    predictions_lines = []
    label_names = np.array(["NONE", "AGAINST", "FAVOR"])
    header = "ID\tTarget\tTweet\tStance\n"
    labels = label_names[predicted_labels]

In [15]:
# accuracy
labels_gr = []

for label in model.labels_test:
    if label == u'NONE':
        labels_gr.append(0)
    elif label == u'AGAINST':
        labels_gr.append(1)
    elif label == u'FAVOR':
        labels_gr.append(2)
        
labels_pr = []
        
for label in labels:
    if label == u'NONE':
        labels_pr.append(0)
    elif label == u'AGAINST':
        labels_pr.append(1)
    elif label == u'FAVOR':
        labels_pr.append(2)

accuracies = []
for idx in xrange(len(labels_pr)):
    if labels_pr[idx] == labels_gr[idx]:
        accuracies.append(1)
    else:
        accuracies.append(0)
accuracy = float(sum(accuracies)) / len(accuracies)
print(accuracy)

0.635077793494


In [17]:
res1 = [0.629420084866, 0.625176803395, 0.639321074965, 0.630834512023, 0.635077793494]

In [22]:
np.mean(res1)

0.63196605374859993

In [23]:
np.std(res1)

0.0048504747381796507